In [ ]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 48 kB/s 
     |████████████████████████████████| 199 kB 67.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=9883232b991e608ab8fcac4c515c36ac41b11eedad10249b1719bd8438ec4567
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc.version

'3.3.1'

In [16]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm") \
    .load("/content/drive/MyDrive/dataset/sample_libsvm_data.txt")
  
# Split the data into train and test
splits = data.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)

# select example rows to display.
predictions = model.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+-----+--------------------+--------------------+-----------+----------+
|label|            features|       rawPrediction|probability|prediction|
+-----+--------------------+--------------------+-----------+----------+
|  0.0|(692,[98,99,100,1...|[-176798.45359013...|  [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,124...|[-189158.41593609...|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[-210439.09448849...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-171124.40702548...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-213017.59308161...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-183431.70141377...|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|[-246449.63384784...|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|[-144270.60315757...|  [1.0,0.0]|       0.0|
|  1.0|(692,[100,101,102...|[-144997.52281187...|  [0.0,1.0]|       1.0|
|  1.0|(692,[123,124,125...|[-138655.43253299...|  [0.0,1.0]|       1.0|
|  1.0|(692,[124,125,126...|[-128503.73534101...|  

In [12]:
asdf = train.collect()
asdf

[Row(label=0.0, features=SparseVector(692, {95: 56.0, 96: 247.0, 97: 121.0, 123: 24.0, 124: 242.0, 125: 245.0, 126: 122.0, 152: 231.0, 153: 253.0, 154: 253.0, 155: 104.0, 156: 12.0, 180: 90.0, 181: 253.0, 182: 253.0, 183: 254.0, 184: 221.0, 185: 120.0, 186: 120.0, 187: 85.0, 205: 67.0, 206: 75.0, 207: 36.0, 208: 11.0, 209: 56.0, 210: 222.0, 211: 254.0, 212: 253.0, 213: 253.0, 214: 253.0, 215: 245.0, 216: 207.0, 217: 36.0, 232: 86.0, 233: 245.0, 234: 249.0, 235: 105.0, 238: 44.0, 239: 224.0, 240: 230.0, 241: 253.0, 242: 253.0, 243: 253.0, 244: 253.0, 245: 214.0, 246: 10.0, 259: 8.0, 260: 191.0, 261: 253.0, 262: 143.0, 268: 29.0, 269: 119.0, 270: 119.0, 271: 158.0, 272: 253.0, 273: 253.0, 274: 94.0, 287: 15.0, 288: 253.0, 289: 226.0, 290: 48.0, 299: 4.0, 300: 183.0, 301: 253.0, 302: 248.0, 303: 56.0, 315: 42.0, 316: 253.0, 317: 178.0, 328: 179.0, 329: 253.0, 330: 184.0, 331: 14.0, 343: 164.0, 344: 253.0, 345: 178.0, 356: 179.0, 357: 253.0, 358: 163.0, 370: 61.0, 371: 254.0, 372: 254.0, 3

In [18]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm")\
    .load("/content/drive/MyDrive/dataset/sample_multiclass_classification_data.txt")

# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [4, 5, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(train)

# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.9523809523809523
